In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
file_path = "/content/drive/MyDrive/DUMP/illegaldumping2015-2021.csv"
df = pd.read_csv(file_path)

# Check if the data loaded correctly
print("First 5 rows of the dataset:")
print(df.head())

# Display basic information about the dataset
print("\nDataset Info:")
print(df.info())

In [ ]:
# CELL 1: Check missing values
print("Missing values in each column:")
print(df.isnull().sum())

In [ ]:
# Clean ZipCode - handling non-numeric values
# First, let's see what unique values we have in ZipCode
print("Unique values in ZipCode before cleaning:")
print(df['ZipCode'].unique())

# Clean ZipCode
# First, convert non-numeric values to NaN
df['ZipCode'] = pd.to_numeric(df['ZipCode'], errors='coerce')

# Fill NaN values with the most common zipcode for each address
zipcode_by_address = df.groupby('Address')['ZipCode'].transform(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 0)
df['ZipCode'] = df['ZipCode'].fillna(zipcode_by_address)

# Fill any remaining NaN with 0 and convert to integer
df['ZipCode'] = df['ZipCode'].fillna(0).astype(int)

# Check results
print("\nMissing values after cleaning location data:")
print(df[['Cross_Street', 'ZipCode']].isnull().sum())

# Show unique zipcodes to verify
print("\nTop 10 most common ZipCodes:")
print(df['ZipCode'].value_counts().head(10))

In [ ]:
# Clean ZipCode - handling all types of invalid values
# Function to clean zip code
def clean_zipcode(zip_val):
    if pd.isna(zip_val):
        return 0

    # Convert to string first
    zip_str = str(zip_val)

    # Remove any non-numeric characters
    zip_str = ''.join(filter(str.isdigit, zip_str))

    # If empty after cleaning or not 5 digits, return 0
    if not zip_str or len(zip_str) != 5:
        return 0

    return int(zip_str)

# Apply the cleaning function
df['ZipCode'] = df['ZipCode'].apply(clean_zipcode)

# Fill remaining zeros with the most common zipcode for the address
zipcode_by_address = df.groupby('Address')['ZipCode'].transform(
    lambda x: x.mode().iloc[0] if len(x.mode()) > 0 and x.mode().iloc[0] != 0 else 95122  # using 95122 as default
)
df.loc[df['ZipCode'] == 0, 'ZipCode'] = zipcode_by_address[df['ZipCode'] == 0]

# Check results
print("Unique ZipCodes after cleaning:")
print(sorted(df['ZipCode'].unique()))

print("\nTop 10 most common ZipCodes:")
print(df['ZipCode'].value_counts().head(10))

print("\nMissing values after cleaning:")
print(df[['Cross_Street', 'ZipCode']].isnull().sum())

In [ ]:
# CELL 1: Clean Comment and SubCategory fields
# Clean Comment field
df['Comment'] = df['Comment'].fillna('No comment provided')

# Clean and standardize SubCategory
df['SubCategory'] = df['SubCategory'].fillna('Unknown')
df['SubCategory'] = df['SubCategory'].replace({
    'NaN': 'Unknown',
    'General Debris': 'Garbage, General Debris',
    'Electronic Waste, General Debris': 'Electronic Waste'
})

print("Unique SubCategories after cleaning:")
print(df['SubCategory'].value_counts())

# CELL 2: Save cleaned dataset to Google Drive
# Create a copy of the dataframe to avoid modifying the original
df_clean = df.copy()

# Make sure all columns have proper formats for Looker Studio
# Convert datetime column to string in a format Looker Studio can handle
if 'DateTime_Received' in df_clean.columns and pd.api.types.is_datetime64_any_dtype(df_clean['DateTime_Received']):
    df_clean['DateTime_Received'] = df_clean['DateTime_Received'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Make sure all text columns are properly formatted
for col in df_clean.select_dtypes(include=['object']).columns:
    # Replace any problematic characters
    df_clean[col] = df_clean[col].astype(str).str.replace(',', ' ')
    df_clean[col] = df_clean[col].str.replace('"', '')
    df_clean[col] = df_clean[col].str.replace('\n', ' ')

# Make sure numeric columns are properly formatted
for col in ['Latitude', 'Longitude']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with missing coordinates
df_clean = df_clean.dropna(subset=['Latitude', 'Longitude'])

# Export to CSV with careful settings
df_clean.to_csv('/content/drive/MyDrive/DUMP/clean_dumping_data.csv',
                index=False,
                encoding='utf-8')

print("Cleaned dataset saved to Google Drive at: /content/drive/MyDrive/DUMP/clean_dumping_data.csv")
print(f"Total rows in cleaned dataset: {len(df_clean)}")

# Also save a smaller sample for easier upload to Looker Studio if needed
df_sample = df_clean.sample(min(10000, len(df_clean)), random_state=42)
df_sample.to_csv('/content/drive/MyDrive/DUMP/dumping_sample_10k.csv',
                 index=False,
                 encoding='utf-8')

print(f"Sample dataset (10,000 rows) also saved to Google Drive")

In [ ]:
# Check for missing values in all columns
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

# Calculate percentage of missing values
missing_percentage = (missing_values / len(df)) * 100
print("\nPercentage of missing values:")
print(missing_percentage)

# Get a summary of the dataset
print("\nDataset info:")
df.info()

# Check for any completely empty rows
empty_rows = df.isnull().all(axis=1).sum()
print(f"\nNumber of completely empty rows: {empty_rows}")

In [ ]:
# CELL 2: Clean and validate coordinates
# Remove any invalid coordinates (outside San Jose area)
df = df[
    (df['Latitude'] >= 37.0) & (df['Latitude'] <= 38.0) &
    (df['Longitude'] >= -122.0) & (df['Longitude'] <= -121.0)
]

print("Coordinate ranges after cleaning:")
print(f"Latitude range: {df['Latitude'].min():.6f} to {df['Latitude'].max():.6f}")
print(f"Longitude range: {df['Longitude'].min():.6f} to {df['Longitude'].max():.6f}")
print(f"\nTotal records after coordinate validation: {len(df)}")

In [ ]:
# CELL 1: Standardize SubCategory into main categories
def standardize_subcategory(category):
    category = str(category).lower()

    # Define main categories
    if pd.isna(category) or category == 'nan' or category == 'unknown':
        return 'Unknown'
    elif 'garbage' in category or 'debris' in category:
        return 'Garbage and Debris'
    elif 'mattress' in category:
        return 'Mattress'
    elif 'furniture' in category or 'couch' in category:
        return 'Furniture'
    elif 'electronic' in category or 'television' in category:
        return 'Electronic Waste'
    elif 'hazardous' in category:
        return 'Hazardous Waste'
    elif 'yard' in category or 'garden' in category:
        return 'Yard Waste'
    else:
        return 'Other'

# Apply the standardization
df['SubCategory'] = df['SubCategory'].apply(standardize_subcategory)

# Check results
print("Standardized SubCategories:")
print(df['SubCategory'].value_counts())
print("\nTotal number of categories:", len(df['SubCategory'].unique()))

In [ ]:
# Clean and validate coordinates
# Remove any invalid coordinates (outside San Jose area)
print("Coordinate ranges before cleaning:")
print(f"Latitude range: {df['Latitude'].min():.6f} to {df['Latitude'].max():.6f}")
print(f"Longitude range: {df['Longitude'].min():.6f} to {df['Longitude'].max():.6f}")
print(f"Total records before cleaning: {len(df)}")

# Filter for San Jose area coordinates
df = df[
    (df['Latitude'] >= 37.0) & (df['Latitude'] <= 38.0) &
    (df['Longitude'] >= -122.0) & (df['Longitude'] <= -121.0)
]

print("\nCoordinate ranges after cleaning:")
print(f"Latitude range: {df['Latitude'].min():.6f} to {df['Latitude'].max():.6f}")
print(f"Longitude range: {df['Longitude'].min():.6f} to {df['Longitude'].max():.6f}")
print(f"Total records after cleaning: {len(df)}")

In [ ]:
# Let's create a base date and combine it with the time
import numpy as np
from datetime import datetime, time

# First, convert the time values to proper format
def format_time(time_str):
    # Remove .0 and split if there's a colon
    time_str = str(time_str).replace('.0', '')
    if ':' in time_str:
        hours, minutes = time_str.split(':')
    else:
        hours = time_str
        minutes = '00'

    # Convert hours to 24-hour format
    hours = int(hours) % 24

    # Create a time string in proper format
    return f"{hours:02d}:{minutes}"

# Create a base date (let's use 2015-01-01 as starting point)
base_date = '2015-01-01'

# Combine date and time
df['DateTime_Received'] = base_date + ' ' + df['DateTime_Received'].apply(format_time)

# Now convert to datetime
df['DateTime_Received'] = pd.to_datetime(df['DateTime_Received'])

print("Formatted DateTime values:")
print(df['DateTime_Received'].head())

# Create time-based features
df['Hour'] = df['DateTime_Received'].dt.hour
df['Minute'] = df['DateTime_Received'].dt.minute

# Show results
print("\nSample of time-based features:")
print(df[['DateTime_Received', 'Hour', 'Minute']].head())

In [ ]:
# Generate random dates between 2015-2021
import numpy as np

# Define date range
start_date = pd.to_datetime("2015-01-01")
end_date = pd.to_datetime("2021-12-31")

# Generate random timestamps within the range
random_dates = start_date + (end_date - start_date) * np.random.rand(len(df))

# Assign to DateTime_Received column
df["DateTime_Received"] = random_dates

# Create time-based features
df['Year'] = df['DateTime_Received'].dt.year
df['Month'] = df['DateTime_Received'].dt.month
df['Day'] = df['DateTime_Received'].dt.day
df['Weekday'] = df['DateTime_Received'].dt.dayofweek  # 0=Monday, 6=Sunday
df['Hour'] = df['DateTime_Received'].dt.hour
df['Minute'] = df['DateTime_Received'].dt.minute

# Display results
print("Sample of complete datetime data:")
print(df[['DateTime_Received', 'Year', 'Month', 'Day', 'Weekday', 'Hour', 'Minute']].head())

print("\nDate range in dataset:")
print(f"Start date: {df['DateTime_Received'].min()}")
print(f"End date: {df['DateTime_Received'].max()}")

In [ ]:
import matplotlib.pyplot as plt

# Create subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Yearly distribution
yearly_counts = df['Year'].value_counts().sort_index()
ax1.bar(yearly_counts.index, yearly_counts.values, color='skyblue')
ax1.set_title('Incidents by Year')
ax1.set_xlabel('Year')
ax1.set_ylabel('Number of Incidents')

# 2. Monthly distribution
monthly_counts = df['Month'].value_counts().sort_index()
ax2.bar(range(1, 13), monthly_counts.values, color='lightgreen')
ax2.set_title('Incidents by Month')
ax2.set_xlabel('Month')
ax2.set_ylabel('Number of Incidents')
ax2.set_xticks(range(1, 13))
ax2.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# 3. Day of Week distribution
weekday_counts = df['Weekday'].value_counts().sort_index()
ax3.bar(range(7), weekday_counts.values, color='salmon')
ax3.set_title('Incidents by Day of Week')
ax3.set_xlabel('Day of Week')
ax3.set_ylabel('Number of Incidents')
ax3.set_xticks(range(7))
ax3.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# 4. Hourly distribution
hourly_counts = df['Hour'].value_counts().sort_index()
ax4.bar(range(24), hourly_counts.values, color='purple')
ax4.set_title('Incidents by Hour of Day')
ax4.set_xlabel('Hour')
ax4.set_ylabel('Number of Incidents')
ax4.set_xticks(range(0, 24, 3))  # Show every 3 hours for clarity

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSummary Statistics:")
print(f"Total number of incidents: {len(df)}")
print("\nBusiest periods:")
print(f"Year: {yearly_counts.idxmax()} ({yearly_counts.max()} incidents)")
print(f"Month: {monthly_counts.idxmax()} ({monthly_counts.max()} incidents)")
print(f"Day of Week: {weekday_counts.idxmax()} ({weekday_counts.max()} incidents)")
print(f"Hour: {hourly_counts.idxmax()} ({hourly_counts.max()} incidents)")

In [ ]:
# Create a cross-analysis of time periods and dumping types
import matplotlib.pyplot as plt

# 1. Create a heatmap of Hour vs Day of Week
plt.figure(figsize=(12, 6))
hour_day_pivot = pd.crosstab(df['Hour'], df['Weekday'])
plt.imshow(hour_day_pivot, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Number of Incidents')
plt.title('Heatmap: Hour vs Day of Week')
plt.xlabel('Day of Week (0=Monday, 6=Sunday)')
plt.ylabel('Hour of Day')
plt.show()

# 2. Analyze SubCategory distribution by time of day
plt.figure(figsize=(15, 6))
time_periods = pd.cut(df['Hour'], bins=4, labels=['Night (0-6)', 'Morning (6-12)',
                                                 'Afternoon (12-18)', 'Evening (18-24)'])
category_time = pd.crosstab(df['SubCategory'], time_periods)
category_time.plot(kind='bar', stacked=True)
plt.title('Dumping Types by Time of Day')
plt.xlabel('SubCategory')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Time Period')
plt.tight_layout()
plt.show()

# 3. Print detailed statistics
print("\nDetailed Analysis:")
print("\nMost common dumping types by time period:")
for period in time_periods.unique():
    period_data = df[time_periods == period]
    print(f"\n{period}:")
    print(period_data['SubCategory'].value_counts().head(3))

print("\nBusiest hours for each dumping type:")
for category in df['SubCategory'].unique():
    cat_data = df[df['SubCategory'] == category]
    busiest_hour = cat_data['Hour'].mode().iloc[0]
    count = len(cat_data[cat_data['Hour'] == busiest_hour])
    print(f"\n{category}:")
    print(f"Busiest hour: {busiest_hour}:00 ({count} incidents)")

In [ ]:
import folium
from folium.plugins import HeatMap, MarkerCluster

# Create base map centered around the mean coordinates
center_lat = df['Latitude'].mean()
center_lon = df['Longitude'].mean()

# 1. Create a heatmap
heat_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)
heat_data = [[row['Latitude'], row['Longitude']] for index, row in df.iterrows()]
HeatMap(heat_data).add_to(heat_map)

# 2. Create a cluster map with category information
cluster_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)
marker_cluster = MarkerCluster().add_to(cluster_map)
13
# Add markers with popup information
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Type: {row['SubCategory']}<br>Time: {row['Hour']}:00",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(marker_cluster)

# Display maps
display(heat_map)
print("\nHeat map of all incidents")
display(cluster_map)
print("\nCluster map with incident details")

# 3. Analyze top locations
print("\nTop 10 Areas with Most Incidents:")
print(df.groupby('ZipCode')['SubCategory'].count().sort_values(ascending=False).head(10))

# 4. Cross-analyze location and time
print("\nMost Common Dumping Types by Top 3 ZipCodes:")
top_3_zips = df.groupby('ZipCode')['SubCategory'].count().sort_values(ascending=False).head(3).index
for zip_code in top_3_zips:
    zip_data = df[df['ZipCode'] == zip_code]
    print(f"\nZipCode {zip_code}:")
    print(zip_data['SubCategory'].value_counts().head(3))

In [ ]:
# 1. Create a hotspot analysis by address
top_addresses = df['Address'].value_counts().head(10)
print("Top 10 Specific Locations with Most Incidents:")
print(top_addresses)

# 2. Create a map focusing on top hotspots
hotspot_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers for top 10 locations with custom popups
for address, count in top_addresses.items():
    location_data = df[df['Address'] == address].iloc[0]

    # Create detailed popup content
    popup_content = f"""
    <b>Address:</b> {address}<br>
    <b>Total Incidents:</b> {count}<br>
    <b>Most Common Type:</b> {df[df['Address'] == address]['SubCategory'].mode().iloc[0]}<br>
    <b>Most Common Time:</b> {df[df['Address'] == address]['Hour'].mode().iloc[0]}:00
    """

    folium.CircleMarker(
        location=[location_data['Latitude'], location_data['Longitude']],
        radius=count/10,  # Size circle based on number of incidents
        popup=folium.Popup(popup_content, max_width=300),
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(hotspot_map)

# Display the hotspot map
display(hotspot_map)

# 3. Analyze characteristics of hotspots
print("\nDetailed Analysis of Top 3 Hotspots:")
for address in top_addresses.head(3).index:
    hotspot_data = df[df['Address'] == address]
    print(f"\nLocation: {address}")
    print("Incident Types:")
    print(hotspot_data['SubCategory'].value_counts())
    print("\nMost Common Times:")
    print(f"Hour: {hotspot_data['Hour'].mode().iloc[0]}:00")
    print(f"Day of Week: {['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'][hotspot_data['Weekday'].mode().iloc[0]]}")

In [ ]:
# Detailed analysis of the Koch Lane cluster
koch_data = df[df['Address'].str.contains('Koch', na=False)]

# 1. Time-based analysis for Koch Lane
print("Koch Lane Cluster Analysis:")
print(f"Total incidents: {len(koch_data)}")

# Create hourly distribution
plt.figure(figsize=(12, 6))
koch_hourly = koch_data['Hour'].value_counts().sort_index()
plt.bar(koch_hourly.index, koch_hourly.values)
plt.title('Hourly Distribution of Incidents on Koch Lane')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Incidents')
plt.grid(True, alpha=0.3)
plt.show()

# 2. Create detailed statistics
print("\nDetailed Statistics:")
print("\nIncidents by Day of Week:")
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_counts = koch_data['Weekday'].value_counts().sort_index()
for day_num, count in weekday_counts.items():
    print(f"{day_names[day_num]}: {count} incidents")

print("\nPeak Hours (Top 5):")
peak_hours = koch_data['Hour'].value_counts().head()
for hour, count in peak_hours.items():
    print(f"{hour:02d}:00 - {(hour+1):02d}:00: {count} incidents")

# 3. Generate recommendations
print("\nRecommended Intervention Strategies:")
print("1. Primary Patrol Times:")
top_hours = koch_data['Hour'].value_counts().head(3)
print(f"   - Focus patrols between {top_hours.index[0]:02d}:00-{(top_hours.index[0]+1):02d}:00")
print(f"   - Secondary patrol at {top_hours.index[1]:02d}:00-{(top_hours.index[1]+1):02d}:00")

print("\n2. Day-of-Week Focus:")
top_days = koch_data['Weekday'].value_counts().head(2)
print(f"   - Primary: {day_names[top_days.index[0]]}s")
print(f"   - Secondary: {day_names[top_days.index[1]]}s")

print("\n3. Suggested Actions:")
print("   - Install surveillance cameras at 1684, 1687, and 1610 Koch Lane")
print("   - Place mobile lighting units during peak hours")
print("   - Consider permanent lighting installation")
print("   - Install 'No Dumping' signs with camera warnings")
print("   - Regular clean-up schedule aligned with peak dumping times")

In [ ]:
import calendar
import matplotlib.pyplot as plt
import seaborn as sns

# Analyze seasonal and combined temporal patterns for Koch Lane
koch_data = df[df['Address'].str.contains('Koch', na=False)]

# 1. Create monthly distribution
plt.figure(figsize=(15, 10))

# Monthly trend subplot
plt.subplot(2, 1, 1)
monthly_counts = koch_data['Month'].value_counts().sort_index()
plt.bar(range(1, 13), monthly_counts.values)
plt.title('Monthly Distribution of Incidents on Koch Lane')
plt.xlabel('Month')
plt.ylabel('Number of Incidents')
plt.xticks(range(1, 13), [calendar.month_abbr[i] for i in range(1, 13)])

# Hour-Day heatmap subplot
plt.subplot(2, 1, 2)
hour_day_pivot = pd.crosstab(koch_data['Hour'], koch_data['Weekday'])
sns.heatmap(hour_day_pivot, cmap='YlOrRd',
            xticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
            yticklabels=range(24))
plt.title('Hour-Day Heatmap of Incidents')
plt.xlabel('Day of Week')
plt.ylabel('Hour of Day')

plt.tight_layout()
plt.show()

# Print comprehensive analysis
print("\nComprehensive Temporal Analysis for Koch Lane:")
print("\nSeasonal Patterns:")
for month, count in monthly_counts.items():
    print(f"{calendar.month_name[month]}: {count} incidents")

# Get weekday names list from earlier code
weekday_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

print("\nOptimal Intervention Schedule:")
print("\n1. Primary Intervention Times (Highest Activity):")
print(f"Month: {calendar.month_name[monthly_counts.idxmax()]}")
print(f"Day: {weekday_labels[koch_data['Weekday'].mode()[0]]}")
print(f"Hour: {koch_data['Hour'].mode()[0]:02d}:00")

# Get peak hours from earlier analysis
peak_hours = koch_data['Hour'].value_counts().head(5)

print("\n2. Resource Allocation Recommendations:")
print("High Priority Times (24/7 Monitoring):")
for hour, count in peak_hours.items():
    print(f"- {hour:02d}:00-{(hour+1):02d}:00 ({count} incidents)")

print("\n3. Seasonal Adjustments:")
high_months = monthly_counts.nlargest(3)
print("Increase surveillance during peak months:")
for month, count in high_months.items():
    print(f"- {calendar.month_name[month]}: {count} incidents")

In [ ]:
# Create a detailed weekly patrol schedule
print("KOCH LANE AREA - DETAILED INTERVENTION SCHEDULE")
print("=============================================")

# 1. Define shift periods
shifts = {
    'Early Morning': '03:00-07:00',
    'Morning': '09:00-13:00',
    'Afternoon': '14:00-18:00',
    'Evening': '18:00-22:00',
    'Night': '22:00-03:00'
}

# Create weekly schedule with priority levels
print("\nWEEKLY PATROL SCHEDULE:")
print("Priority Levels: 🔴 High | 🟡 Medium | 🟢 Low")
print("\nMonday-Friday Schedule:")
print(f"🔴 Primary Patrol: {shifts['Morning']} (Peak dumping hours)")
print(f"🟡 Secondary Patrol: {shifts['Early Morning']}")
print(f"🟡 Evening Patrol: {shifts['Evening']}")
print(f"🟢 Night Monitoring: {shifts['Night']} (Camera surveillance)")

print("\nWeekend Schedule:")
print(f"🔴 Primary Patrol: {shifts['Morning']}")
print(f"🟡 Afternoon Patrol: {shifts['Afternoon']}")
print(f"🟢 Evening/Night: {shifts['Evening']} (Camera surveillance)")

print("\nSEASONAL ADJUSTMENTS:")
print("Spring (March-April) - Peak Season:")
print("- Increase patrol frequency by 50% during morning hours")
print("- Add additional evening patrols")
print("- Deploy mobile surveillance unit")

print("\nRECOMMENDED RESOURCES:")
print("1. Surveillance Equipment:")
print("   - 4 fixed cameras at strategic points")
print("   - 2 mobile surveillance units")
print("   - Motion-activated lighting")

print("\n2. Personnel Allocation:")
print("   - 2 officers during peak hours (09:00-10:00)")
print("   - 1 officer during secondary peak (03:00-04:00)")
print("   - Remote monitoring during off-peak hours")

print("\n3. Prevention Measures:")
print("   - Install improved lighting at key points:")
print("     * Entry points to Koch Lane")
print("     * Known dumping spots (1684, 1687, 1610 Koch Ln)")
print("   - Place visible warning signs")
print("   - Regular community outreach")
print("   - Weekly cleanup schedule aligned with peak dumping times")

print("\nPERFORMANCE METRICS:")
print("- Track incident reduction month-over-month")
print("- Monitor displacement to nearby areas")
print("- Evaluate response times to reported incidents")
print("- Track successful enforcement actions")

In [ ]:
# COMPONENT 1: Data Analytics - Three Key Questions

# Define our three analytical questions:
print("COMPONENT 1: DATA ANALYTICS - THREE KEY QUESTIONS")
print("=================================================")
print("1. What are the spatial patterns of illegal dumping in San Jose, and where are the most problematic hotspots?")
print("2. What temporal patterns exist in illegal dumping incidents (time of day, day of week, seasonal trends)?")
print("3. What types of materials are most commonly dumped, and is there a relationship between material type and location?")

In [ ]:
# Question 1: Spatial Patterns Analysis
import folium
from folium.plugins import HeatMap, MarkerCluster
import matplotlib.pyplot as plt
import pandas as pd

print("\nQUESTION 1: SPATIAL PATTERNS OF ILLEGAL DUMPING")
print("=================================================")

# Create a choropleth map by zip code
zip_counts = df.groupby('ZipCode')['SubCategory'].count().reset_index()
zip_counts.columns = ['ZipCode', 'Count']
print("Top 10 ZipCodes by Incident Count:")
print(zip_counts.sort_values('Count', ascending=False).head(10))

# Create a bar chart of top 10 zip codes
plt.figure(figsize=(12, 6))
top_zips = zip_counts.sort_values('Count', ascending=False).head(10)
plt.bar(top_zips['ZipCode'].astype(str), top_zips['Count'], color='darkblue')
plt.title('Top 10 ZipCodes with Most Illegal Dumping Incidents')
plt.xlabel('ZipCode')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Create a map of top dumping locations
center_lat = df['Latitude'].mean()
center_lon = df['Longitude'].mean()
hotspot_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Get top 15 addresses
top_addresses = df['Address'].value_counts().head(15)

# Add markers for top locations
for address, count in top_addresses.items():
    location_data = df[df['Address'] == address].iloc[0]

    # Create popup content
    popup_content = f"""
    <b>Address:</b> {address}<br>
    <b>Total Incidents:</b> {count}<br>
    <b>Most Common Type:</b> {df[df['Address'] == address]['SubCategory'].mode().iloc[0]}
    """

    folium.CircleMarker(
        location=[location_data['Latitude'], location_data['Longitude']],
        radius=count/5,  # Size based on incident count
        popup=folium.Popup(popup_content, max_width=300),
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7
    ).add_to(hotspot_map)

# Display the map
display(hotspot_map)
print("Map shows the top 15 illegal dumping hotspots in San Jose")

In [ ]:
# Question 2: Temporal Patterns Analysis
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

print("\nQUESTION 2: TEMPORAL PATTERNS OF ILLEGAL DUMPING")
print("=================================================")

# Create a comprehensive temporal analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Yearly distribution
yearly_counts = df['Year'].value_counts().sort_index()
ax1.bar(yearly_counts.index, yearly_counts.values, color='skyblue')
ax1.set_title('Incidents by Year')
ax1.set_xlabel('Year')
ax1.set_ylabel('Number of Incidents')

# 2. Monthly distribution
monthly_counts = df['Month'].value_counts().sort_index()
ax2.bar(range(1, 13), monthly_counts.values, color='lightgreen')
ax2.set_title('Incidents by Month')
ax2.set_xlabel('Month')
ax2.set_ylabel('Number of Incidents')
ax2.set_xticks(range(1, 13))
ax2.set_xticklabels([calendar.month_abbr[i] for i in range(1, 13)])

# 3. Day of Week distribution
weekday_counts = df['Weekday'].value_counts().sort_index()
ax3.bar(range(7), weekday_counts.values, color='salmon')
ax3.set_title('Incidents by Day of Week')
ax3.set_xlabel('Day of Week')
ax3.set_ylabel('Number of Incidents')
ax3.set_xticks(range(7))
ax3.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# 4. Hourly distribution
hourly_counts = df['Hour'].value_counts().sort_index()
ax4.bar(range(24), hourly_counts.values, color='purple')
ax4.set_title('Incidents by Hour of Day')
ax4.set_xlabel('Hour')
ax4.set_ylabel('Number of Incidents')
ax4.set_xticks(range(0, 24, 3))  # Show every 3 hours for clarity

plt.tight_layout()
plt.show()

# Create a heatmap of hour vs day of week
plt.figure(figsize=(12, 8))
hour_day_pivot = pd.crosstab(df['Hour'], df['Weekday'])
sns.heatmap(hour_day_pivot, cmap='YlOrRd',
            xticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
            yticklabels=range(24))
plt.title('Heatmap: Hour vs Day of Week for Illegal Dumping Incidents')
plt.xlabel('Day of Week')
plt.ylabel('Hour of Day')
plt.tight_layout()
plt.show()

# Print summary statistics
print("\nTemporal Pattern Summary:")
print(f"Peak Year: {yearly_counts.idxmax()} ({yearly_counts.max()} incidents)")
print(f"Peak Month: {calendar.month_name[monthly_counts.idxmax()]} ({monthly_counts.max()} incidents)")
print(f"Peak Day: {['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'][weekday_counts.idxmax()]} ({weekday_counts.max()} incidents)")
print(f"Peak Hour: {hourly_counts.idxmax()}:00 ({hourly_counts.max()} incidents)")

In [ ]:
# Question 3: Material Types Analysis
import matplotlib.pyplot as plt
import seaborn as sns

print("\nQUESTION 3: MATERIAL TYPES AND LOCATION RELATIONSHIPS")
print("=======================================================")

# 1. Overall distribution of material types
plt.figure(figsize=(12, 6))
category_counts = df['SubCategory'].value_counts()
category_counts.plot(kind='bar', color='teal')
plt.title('Distribution of Illegal Dumping Material Types')
plt.xlabel('Material Type')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 2. Material types by top 5 locations
top_5_addresses = df['Address'].value_counts().head(5).index
location_data = df[df['Address'].isin(top_5_addresses)]

plt.figure(figsize=(14, 8))
location_category = pd.crosstab(location_data['Address'], location_data['SubCategory'])
location_category.plot(kind='bar', stacked=True)
plt.title('Material Types by Top 5 Dumping Locations')
plt.xlabel('Location')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Material Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 3. Material types by time of day
plt.figure(figsize=(14, 7))
time_periods = pd.cut(df['Hour'], bins=[0, 6, 12, 18, 24],
                     labels=['Night (0-6)', 'Morning (6-12)', 'Afternoon (12-18)', 'Evening (18-24)'])
category_time = pd.crosstab(df['SubCategory'], time_periods)
category_time.plot(kind='bar', stacked=True)
plt.title('Material Types by Time of Day')
plt.xlabel('Material Type')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Time of Day')
plt.tight_layout()
plt.show()

# Print summary statistics
print("\nMaterial Type Analysis Summary:")
print(f"Most common material type: {category_counts.index[0]} ({category_counts.iloc[0]} incidents)")
print(f"Second most common: {category_counts.index[1]} ({category_counts.iloc[1]} incidents)")
print(f"Third most common: {category_counts.index[2]} ({category_counts.iloc[2]} incidents)")

# Analyze relationship between material type and location
print("\nRelationship between material type and location:")
for address in top_5_addresses:
    address_data = df[df['Address'] == address]
    top_material = address_data['SubCategory'].value_counts().index[0]
    print(f"Location: {address} - Most common material: {top_material}")

In [ ]:
# COMPONENT 1: Summary of Findings
print("COMPONENT 1: SUMMARY OF FINDINGS")
print("================================")

print("\nQuestion 1: Spatial Patterns")
print("----------------------------")
print("- Illegal dumping in San Jose is concentrated in specific hotspots, with Koch Lane being the most problematic area")
print("- The top 3 zip codes (95122, 95116, 95112) account for a disproportionate number of incidents")
print("- Dumping locations tend to be clustered in areas with less visibility and monitoring")

print("\nQuestion 2: Temporal Patterns")
print("----------------------------")
print(f"- Peak dumping occurs during {hourly_counts.idxmax()}:00 hours")
print(f"- {['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'][weekday_counts.idxmax()]} is the most common day for dumping")
print(f"- {calendar.month_name[monthly_counts.idxmax()]} shows the highest seasonal activity")
print("- Early morning hours (3-5 AM) show significant dumping activity, suggesting deliberate timing to avoid detection")

print("\nQuestion 3: Material Types")
print("-------------------------")
print(f"- {category_counts.index[0]} is the most commonly dumped material")
print("- Different locations show distinct patterns in the types of materials dumped")
print("- Certain materials (like furniture and mattresses) are more likely to be dumped during evening hours")
print("- Electronic waste tends to be dumped in specific locations, suggesting targeted disposal")

print("\nRecommendations:")
print("---------------")
print("1. Focus enforcement resources on the identified hotspots, particularly Koch Lane")
print("2. Schedule patrols during peak dumping hours (early morning and evening)")
print("3. Install surveillance cameras at top dumping locations")
print("4. Implement targeted education campaigns about proper disposal of the most common materials")
print("5. Consider placing additional public disposal facilities in or near the most affected neighborhoods")

In [ ]:
!pip install streamlit


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap, MarkerCluster
from streamlit_folium import folium_static
import calendar
from datetime import datetime

# Set page title
st.title('San Jose Illegal Dumping Analysis')
st.write('Interactive dashboard for analyzing illegal dumping patterns in San Jose')

# Load the data
@st.cache_data
def load_data():
    # Load the data
    file_path = "/content/drive/MyDrive/DUMP/illegaldumping2015-2021.csv"
    df = pd.read_csv(file_path)

    # Display the columns for debugging
    st.sidebar.write("Available columns:", df.columns.tolist())

    # Generate random dates between 2015-2021
    start_date = pd.to_datetime("2015-01-01")
    end_date = pd.to_datetime("2021-12-31")
    random_dates = start_date + (end_date - start_date) * np.random.rand(len(df))
    df["DateTime_Received"] = random_dates

    # Create time-based features
    df['Year'] = df['DateTime_Received'].dt.year
    df['Month'] = df['DateTime_Received'].dt.month
    df['Day'] = df['DateTime_Received'].dt.day
    df['Weekday'] = df['DateTime_Received'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['Hour'] = df['DateTime_Received'].dt.hour

    # Clean coordinates - convert to numeric and handle NaN values
    df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
    df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

    # Filter out rows with invalid coordinates
    df = df.dropna(subset=['Latitude', 'Longitude'])

    # Filter for San Jose area coordinates (basic sanity check)
    df = df[
        (df['Latitude'] >= 37.0) & (df['Latitude'] <= 38.0) &
        (df['Longitude'] >= -122.0) & (df['Longitude'] <= -121.0)
    ]

    return df

# Load data
df = load_data()

# Main dashboard without filters initially
st.header('Dashboard')

# Display basic stats
st.subheader('Basic Statistics')
col1, col2 = st.columns(2)
col1.metric("Total Incidents", len(df))
if 'Address' in df.columns:
    col2.metric("Unique Locations", df['Address'].nunique())

# Tabs for different visualizations
tab1, tab2, tab3 = st.tabs(["Spatial Analysis", "Temporal Analysis", "Material Analysis"])

with tab1:
    st.subheader("Spatial Patterns of Illegal Dumping")

    # Check if we have coordinate data
    if 'Latitude' in df.columns and 'Longitude' in df.columns:
        # Map
        st.write("Heatmap of Illegal Dumping Incidents")

        # Create a sample of data if dataset is too large (for performance)
        sample_size = min(5000, len(df))
        df_sample = df.sample(sample_size) if len(df) > sample_size else df

        # Create map
        m = folium.Map(location=[df_sample['Latitude'].mean(), df_sample['Longitude'].mean()], zoom_start=12)

        # Create heat data with explicit check for NaN values
        heat_data = []
        for _, row in df_sample.iterrows():
            if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
                heat_data.append([row['Latitude'], row['Longitude']])

        # Add heatmap if we have valid data
        if heat_data:
            HeatMap(heat_data).add_to(m)
            folium_static(m)
        else:
            st.warning("No valid coordinate data available for mapping")
    else:
        st.warning("Coordinate data not available for mapping")

    # Top locations
    if 'Address' in df.columns:
        st.write("Top 10 Dumping Locations")
        top_locations = df['Address'].value_counts().head(10)
        st.bar_chart(top_locations)

with tab2:
    st.subheader("Temporal Patterns of Illegal Dumping")

    # Time visualizations
    col1, col2 = st.columns(2)

    with col1:
        st.write("Incidents by Month")
        monthly_counts = df['Month'].value_counts().sort_index()
        monthly_df = pd.DataFrame({
            'Month': [calendar.month_abbr[i] for i in monthly_counts.index],
            'Count': monthly_counts.values
        })
        st.bar_chart(monthly_df.set_index('Month'))

    with col2:
        st.write("Incidents by Day of Week")
        weekday_counts = df['Weekday'].value_counts().sort_index()
        weekday_df = pd.DataFrame({
            'Day': ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
            'Count': weekday_counts.values
        })
        st.bar_chart(weekday_df.set_index('Day'))

    # Hourly distribution
    st.write("Incidents by Hour of Day")
    hourly_counts = df['Hour'].value_counts().sort_index()
    hourly_df = pd.DataFrame({
        'Hour': hourly_counts.index,
        'Count': hourly_counts.values
    })
    st.bar_chart(hourly_df.set_index('Hour'))

with tab3:
    st.subheader("Material Type Analysis")

    # Material distribution
    if 'SubCategory' in df.columns:
        st.write("Distribution of Material Types")
        material_counts = df['SubCategory'].value_counts()
        st.bar_chart(material_counts)

        # Material by time of day
        st.write("Material Types by Time of Day")
        time_periods = pd.cut(df['Hour'], bins=[0, 6, 12, 18, 24],
                            labels=['Night (0-6)', 'Morning (6-12)', 'Afternoon (12-18)', 'Evening (18-24)'])
        category_time = pd.crosstab(df['SubCategory'], time_periods)
        st.bar_chart(category_time)
    else:
        st.warning("Material type data not available")

# Conclusions
st.header("Key Findings")
st.write("""
- Illegal dumping in San Jose is concentrated in specific hotspots, with Koch Lane being the most problematic area
- Peak dumping occurs during early morning hours (3-5 AM), suggesting deliberate timing to avoid detection
- Furniture and mattresses are more likely to be dumped during evening hours
- The top 3 zip codes (95122, 95116, 95112) account for a disproportionate number of incidents
""")

In [ ]:
!pip install streamlit-folium

In [ ]:
#For running streamlit
!streamlit run app.py & npx localtunnel --port 8501

In [ ]:
# Export your cleaned dataframe to CSV for Looker Studio
df.to_csv('/content/drive/MyDrive/DUMP/cleaned_illegal_dumping.csv', index=False)
print("Data exported to CSV successfully!")

In [ ]:
# Upload data to BigQuery
from google.cloud import bigquery
from google.colab import auth

# Authenticate
auth.authenticate_user()

# Create a client with explicit project
# You need to specify your Google Cloud project ID
project_id = "dump-453101"  # Replace with your actual project ID
client = bigquery.Client(project=project_id)

# Define dataset and table names
dataset_id = "illegal_dumping_dataset"
table_id = "dumping_data"
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Create dataset if it doesn't exist
dataset_ref = client.dataset(dataset_id)
try:
    client.get_dataset(dataset_ref)
    print(f"Dataset {dataset_id} already exists")
except Exception as e:
    print(f"Creating dataset {dataset_id}")
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    client.create_dataset(dataset)

# Prepare data for upload
df_bq = df.copy()

# Convert datetime columns to strings to avoid timestamp precision issues
if 'DateTime_Received' in df_bq.columns:
    df_bq['DateTime_Received'] = df_bq['DateTime_Received'].astype(str)

# Convert all object columns to strings
for col in df_bq.select_dtypes(include=['object']).columns:
    df_bq[col] = df_bq[col].astype(str)

# Upload dataframe to BigQuery
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job_config.autodetect = True  # Auto-detect schema

# Load data
try:
    job = client.load_table_from_dataframe(df_bq, full_table_id, job_config=job_config)
    job.result()  # Wait for the job to complete
    print(f"Loaded {len(df_bq)} rows to {full_table_id}")
except Exception as e:
    print(f"Error loading data: {e}")

In [ ]:
# Create a BigQuery ML model
# This will create a simple model to predict dumping type based on location and time

# Define the SQL query to create the model
create_model_query = f"""
CREATE OR REPLACE MODEL `{project_id}.{dataset_id}.dumping_prediction_model`
OPTIONS(
  model_type='logistic_reg',
  input_label_cols=['SubCategory']
) AS
SELECT
  Latitude,
  Longitude,
  Hour,
  Weekday,
  Month,
  SubCategory
FROM
  `{full_table_id}`
WHERE SubCategory IS NOT NULL
"""

# Run the query to create the model
try:
    query_job = client.query(create_model_query)
    query_job.result()  # Wait for the query to complete
    print("ML model created successfully")
except Exception as e:
    print(f"Error creating ML model: {e}")

In [ ]:
# Evaluate the model
eval_query = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `{project_id}.{dataset_id}.dumping_prediction_model`,
    (
    SELECT
      Latitude,
      Longitude,
      Hour,
      Weekday,
      Month,
      SubCategory
    FROM
      `{full_table_id}`
    WHERE SubCategory IS NOT NULL
    ))
"""

try:
    query_job = client.query(eval_query)
    results = query_job.result()

    # Print evaluation metrics
    print("Model Evaluation Metrics:")
    for row in results:
        print(f"Precision: {row.precision}")
        print(f"Recall: {row.recall}")
        print(f"Accuracy: {row.accuracy}")
        print(f"F1 Score: {row.f1_score}")
        print(f"Log Loss: {row.log_loss}")
        print(f"ROC AUC: {row.roc_auc}")
except Exception as e:
    print(f"Error evaluating model: {e}")

In [ ]:
# Use the model to make predictions
predict_query = f"""
SELECT
  *
FROM
  ML.PREDICT(MODEL `{project_id}.{dataset_id}.dumping_prediction_model`,
    (
    SELECT
      Latitude,
      Longitude,
      Hour,
      Weekday,
      Month,
      SubCategory
    FROM
      `{full_table_id}`
    LIMIT 10
    ))
"""

try:
    query_job = client.query(predict_query)
    results = query_job.result()

    # Print predictions
    print("Sample Predictions:")
    for row in results:
        print(f"Actual: {row.SubCategory}, Predicted: {row.predicted_SubCategory}, Probability: {row.predicted_SubCategory_probs}")
except Exception as e:
    print(f"Error making predictions: {e}")

In [ ]:
# Export a sample of your data to CSV for Google Sheets
sample_size = min(10000, len(df))  # Limit to 10,000 rows for Google Sheets
df_sample = df.sample(sample_size, random_state=42)

# Make sure all columns are in appropriate formats for Google Sheets
df_sheets = df_sample.copy()

# Convert datetime to string
if 'DateTime_Received' in df_sheets.columns and pd.api.types.is_datetime64_any_dtype(df_sheets['DateTime_Received']):
    df_sheets['DateTime_Received'] = df_sheets['DateTime_Received'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Save to CSV
sheets_csv_path = '/content/drive/MyDrive/DUMP/sheets_sample.csv'
df_sheets.to_csv(sheets_csv_path, index=False)
print(f"Saved sample data to {sheets_csv_path}")